# Testing for modules impacted by diet

In [1]:
#load packages
pkgs <- c("mgcv", "lme4", "ggplot2", "vroom", "dplyr", "forcats", "tidyr", "Seurat", "SingleCellExperiment", "gratia", "MASS", "fitdistrplus", "furrr", "tidyverse")
vapply(pkgs, library, logical(1), character.only = TRUE, logical.return = TRUE,
       quietly = TRUE)

This is mgcv 1.8-42. For overview type 'help("mgcv-package")'.


Attaching package: ‘lme4’


The following object is masked from ‘package:nlme’:

    lmList



Attaching package: ‘dplyr’


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack


The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in pl

mgcv                 lme4              ggplot2 
                TRUE                 TRUE                 TRUE 
               vroom                dplyr              forcats 
                TRUE                 TRUE                 TRUE 
               tidyr               Seurat SingleCellExperiment 
                TRUE                 TRUE                 TRUE 
              gratia                 MASS         fitdistrplus 
                TRUE                 TRUE                 TRUE 
               furrr            tidyverse 
                TRUE                 TRUE

In [2]:
# settings for the run
me_dir <- 'wgcna_output/renamed_tables/' #where to find the WGCNA MEs
run_name <- 'ds4_mcs20_pF_kME_cellModEmbed_16h' #the run prefix
group_col <- "diet" # group that will be modelled
cluster_col <- "region" # sample region
mgcv_dir <- paste0(me_dir, 'mgcv/') # where to save the mgcv results
dir.create(mgcv_dir, recursive = TRUE) # creates the mgcv results folder if it doesn't exist
summaries_path <- paste0(mgcv_dir, run_name, '_summaries_f.rds') #save summaries here
models_path <- paste0(mgcv_dir, run_name, '_models.rds') #save models here
memfs_path <- paste0(mgcv_dir, run_name, '_memfs.rds') #save intermediate memfs objects here (only for debugging)
significant_modules <- paste0(mgcv_dir, run_name, '_significant_modules.csv')#where to save the table summarizing significant modules

Warning message in dir.create(mgcv_dir, recursive = TRUE):
“'wgcna_output/renamed_tables/mgcv' already exists”


In [3]:
me = read_csv(paste0(me_dir, 'kME_cellModEmbed.csv')) #read the MEs and inspect them
me %>% head

Rows: 144 Columns: 386
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): data, region, sample
dbl (383): HPF__01, HPF__02, HPF__03, HPF__04, HPF__05, HPF__06, HPF__07, HP...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


data,region,sample,HPF__01,HPF__02,HPF__03,HPF__04,HPF__05,HPF__06,HPF__07,⋯,ZI__39,ZI__40,ZI__41,ZI__42,ZI__43,ZI__44,ZI__45,ZI__46,ZI__47,ZI__48
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dmb,HY,S0010_01_chow_HY,0.05197933,0.03023810,0.1824172,0.1229980,-0.05775205,0.019439584,-0.06125762,⋯,-0.08595527,0.05252190,-0.05822008,0.06567769,0.036785185,0.0102144634,0.10537119,0.11220870,-0.04791984,0.020875004
dmb,HY,S0010_02_chow_HY,0.08876762,0.08017922,0.1459963,0.1170462,-0.03252780,-0.004745673,-0.05191100,⋯,-0.11281907,0.02927096,-0.02412367,0.02603893,0.007287426,-0.0017344272,0.10817653,0.11961064,-0.04316019,0.006892997
dmb,HY,S0010_03_chow_HY,0.07109426,0.02853630,0.1832191,0.1384336,-0.06336754,-0.032457115,-0.06025051,⋯,-0.10252465,0.06811403,-0.04506138,0.01064958,0.021634418,-0.0002412614,0.07374108,0.10598614,-0.02872247,0.025098740
dmb,HY,S0010_04_chow_HY,0.02948085,0.06371331,0.1535754,0.1107229,-0.03764149,0.025134355,-0.05945944,⋯,-0.05751049,0.03972461,-0.01765908,0.05303416,-0.004217487,0.0049177488,0.06565093,0.01816232,-0.03737867,0.014236332
dmb,HY,S0010_05_chow_HY,0.11411558,0.03807216,0.1615871,0.1345847,-0.06702112,0.007702094,-0.05492529,⋯,-0.15425467,0.03670603,-0.01827709,0.01769293,-0.020943932,-0.0073797921,0.09252465,0.09507013,-0.02601267,0.019362943
dmb,HY,S0010_06_chow_HY,0.02021261,0.07189122,0.1562818,0.1019575,-0.03508683,-0.025868732,-0.07505565,⋯,-0.09517898,0.05157318,-0.02545144,0.02872648,-0.025390663,0.0255168968,0.07634631,0.05995981,-0.04782112,0.008990742


In [4]:
meta = read_csv('vsd_meta.csv') # read the metadata and reformat some columns
meta = meta %>%
mutate(diet = factor(diet, levels=c('chow', 'HFD', 'fast'))) %>%
mutate(animal_id = as.character(animal_id)) %>%
mutate(region = factor(region, levels=c("HY", 'HPF', 'SC', 'IC', 'PFC', 'ZI'))) %>%
filter(seq_run %in% c('0010')) %>% #filter down to 16h study
mutate(seq_run = factor(seq_run, levels=c('0010'))) 

meta %>% head

Rows: 144 Columns: 26
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (9): name, filename, seq_run, seq_sample, diet, region, hemisphere, no...
dbl  (15): rna_conc_qbit, box_id, animal_id, diet_kcal, experiment_duration,...
dttm  (2): experiment_start, time_of_euthanasia

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


name,filename,seq_run,seq_sample,rna_conc_qbit,box_id,animal_id,diet,diet_kcal,region,⋯,bw_f,bw_change,bw_change_pct,bw_change_pct_ph,fw_i,fw_f,kcal_intake,power_intake,notes,qc
<chr>,<chr>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
S0010_01_chow_HY,1_Chow_Hyp_both_S80,0010,S80,73.60,34,1,chow,3227,HY,⋯,24.6,1.2,0.051282051,0.0032051282,741.7,734.4,11778.55,736.1594,NA,ok
S0010_02_chow_HY,2_Chow_Hyp_both_S44,0010,S44,27.20,34,2,chow,3227,HY,⋯,26.0,0.1,0.003861004,0.0002388250,741.7,734.4,11778.55,728.5701,NA,ok
S0010_03_chow_HY,3_Chow_Hyp_both_S78,0010,S78,6.36,35,3,chow,3227,HY,⋯,26.4,0.2,0.007633588,0.0004770992,743.5,735.3,13230.70,826.9188,NA,ok
S0010_04_chow_HY,4_Chow_Hyp_both_S25,0010,S25,20.80,35,4,chow,3227,HY,⋯,24.0,0.5,0.021276596,0.0013160781,743.5,735.3,13230.70,818.3938,NA,ok
S0010_05_chow_HY,5_Chow_Hyp_both_S84,0010,S84,20.40,36,5,chow,3227,HY,⋯,24.4,-0.3,-0.012145749,-0.0007591093,713.6,706.4,11617.20,726.0750,NA,ok
S0010_06_chow_HY,6_Chow_Hyp_both_S66,0010,S66,10.60,36,6,chow,3227,HY,⋯,27.0,1.1,0.042471042,0.0026379529,713.6,706.4,11617.20,721.5652,NA,ok


In [5]:
# filters the meta down
# meta is the meta dataframe
# group_col is the perturbation column e.g. - Diet
# groups is a vector with conditions for comparison e.g. - c("chow", "HFD")
# cluster_col is the column encoding clusters in the meta. e.g. - region
# clusters is a vector with clusters to filter to. e.g. - HY
filter_meta <- function (meta, group_col, groups, cluster_col, clusters){
    meta_f <- meta %>% 
        filter(
            get(group_col) %in% groups,
            get(cluster_col) %in% clusters
        )
    return (meta_f)
}

In [6]:
make_sp <- function (me_df) {
    sp <- stringr::str_split_fixed(colnames(me)[4:length(colnames(me))], pattern='__', n=2)
    sp <- as.data.frame(sp)
    sp <- apply(sp, 1, function(x) as.vector(c(x[1], x[2], paste0(x[1], '__', x[2]))))
    sp <- t(sp)
    sp <- as.data.frame(sp)
    colnames(sp)[colnames(sp) == 'V1'] <- 'cluster'
    colnames(sp)[colnames(sp) == 'V2'] <- 'module'
    colnames(sp)[colnames(sp) == 'V3'] <- 'ME_col'
    return (sp)
}

In [7]:
# a simple model
# models the diet (group_col)
fit_mgcv <- function(memf){
    model <- bam(ME ~ get(group_col),
                 data = memf, method = "fREML", discrete = T
                )
    return (model)
}


# a more complex model
# accounts for sequencing run as a random effect
# fit_mgcv <- function(memf){
#     model <- bam(ME ~ get(group_col) +
#                  s(seq_run, bs = "re"), # could also model cages if known
#                  data = memf, method = "fREML", discrete = T
#                 )
#     return (model)
# }

safe_fit_mgcv <- purrr::safely(fit_mgcv) #necessary for parallelization, in case we hit errors

In [8]:
# this is what make_sp produces
sp <- make_sp(me)
head(sp)

,cluster,module,ME_col
,<chr>,<chr>,<chr>
1,HPF,01,HPF__01
2,HPF,02,HPF__02
3,HPF,03,HPF__03
4,HPF,04,HPF__04
5,HPF,05,HPF__05
6,HPF,06,HPF__06


In [9]:
#this is what filter_meta produces
filter_meta(meta, "diet", c('chow', 'HFD'), "region", c("HY"))

name,filename,seq_run,seq_sample,rna_conc_qbit,box_id,animal_id,diet,diet_kcal,region,⋯,bw_f,bw_change,bw_change_pct,bw_change_pct_ph,fw_i,fw_f,kcal_intake,power_intake,notes,qc
<chr>,<chr>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<fct>,<dbl>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
S0010_01_chow_HY,1_Chow_Hyp_both_S80,0010,S80,73.60,34,1,chow,3227,HY,⋯,24.6,1.2,0.051282051,0.0032051282,741.7,734.4,11778.55,736.1594,NA,ok
S0010_02_chow_HY,2_Chow_Hyp_both_S44,0010,S44,27.20,34,2,chow,3227,HY,⋯,26.0,0.1,0.003861004,0.0002388250,741.7,734.4,11778.55,728.5701,NA,ok
S0010_03_chow_HY,3_Chow_Hyp_both_S78,0010,S78,6.36,35,3,chow,3227,HY,⋯,26.4,0.2,0.007633588,0.0004770992,743.5,735.3,13230.70,826.9188,NA,ok
S0010_04_chow_HY,4_Chow_Hyp_both_S25,0010,S25,20.80,35,4,chow,3227,HY,⋯,24.0,0.5,0.021276596,0.0013160781,743.5,735.3,13230.70,818.3938,NA,ok
S0010_05_chow_HY,5_Chow_Hyp_both_S84,0010,S84,20.40,36,5,chow,3227,HY,⋯,24.4,-0.3,-0.012145749,-0.0007591093,713.6,706.4,11617.20,726.0750,NA,ok
S0010_06_chow_HY,6_Chow_Hyp_both_S66,0010,S66,10.60,36,6,chow,3227,HY,⋯,27.0,1.1,0.042471042,0.0026379529,713.6,706.4,11617.20,721.5652,NA,ok
S0010_14_HFD_HY,14_Highfatdiet_Hyp_both_S28,0010,S28,24.40,40,14,HFD,5560,HY,⋯,26.5,0.4,0.015325670,0.0009450568,495.1,489.4,15846.00,977.1429,NA,ok
S0010_15_HFD_HY,15_Highfatdiet_Hyp_both_S18,0010,S18,12.30,41,15,HFD,5560,HY,⋯,26.3,-0.2,-0.007547170,-0.0004716981,495.0,490.1,13622.00,851.3750,OBS. Lost weight.,ok
S0010_16_HFD_HY,16_Highfatdiet_Hyp_both_S85,0010,S85,29.40,41,16,HFD,5560,HY,⋯,24.3,0.1,0.004132231,0.0002556019,495.0,490.1,13622.00,842.5979,NA,ok


In [10]:
# creating splits of the data for parallelization

splits <- list(c('chow', 'fast'),
               c('chow', 'HFD'))

memfs <- c()

for (s in splits){
    for (i in 1:nrow(sp)){
        cluster <- sp[i,]$cluster
        module <- sp[i,]$module
        me_col <- sp[i,]$ME_col
        me_keep_cols <- c('sample', me_col)
        me_f <- me[me_keep_cols]
        colnames(me_f)[2] <- 'ME'
        mf <- filter_meta(meta, group_col, s, cluster_col, c(cluster))
        mf$sample <- mf$name #need to give this same column as data
        mf_cols <- c('sample', 'diet', 'region', 'seq_run') 
        mf_f <- mf[mf_cols]
        mf_f <- mf_f %>%
            mutate(
                diet = droplevels(diet),
                region = droplevels(region),
                seq_run = droplevels(seq_run)
            )
        memf <- plyr::join(me_f, mf_f, by = c('sample'), type='right')
        memf_name <- paste0(s[1], '_', s[2], '-', me_col)
        memfs[[memf_name]] <- memf
    }
}

In [11]:
# what the splicts look like
memfs[1]

sample,ME,diet,region,seq_run
<chr>,<dbl>,<fct>,<fct>,<fct>
S0010_01_chow_HPF,0.068131606,chow,HPF,0010
S0010_02_chow_HPF,-0.004494135,chow,HPF,0010
S0010_03_chow_HPF,0.040145488,chow,HPF,0010
S0010_05_chow_HPF,0.042448492,chow,HPF,0010
S0010_06_chow_HPF,0.143176854,chow,HPF,0010
S0010_07_fast_HPF,0.133601161,fast,HPF,0010
S0010_08_fast_HPF,0.144179144,fast,HPF,0010
S0010_09_fast_HPF,0.082292832,fast,HPF,0010
S0010_10_fast_HPF,-0.306421403,fast,HPF,0010


In [12]:
# saveRDS(memfs, memfs_path)

In [13]:
# This does run in parallel! adjust workers accordingly
plan(multisession, workers = 20)
models <- future_map(memfs, ~safe_fit_mgcv(.x), .options = furrr_options(seed = T))

Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smooths, ignoring `discrete=TRUE'”
Warning message in bam(ME ~ get(group_col), data = memf, method = "fREML", discrete = T):
“no smo

In [14]:
# function to drop models that error
filter_errored_models <- function(models){
    models_f <- c()
    models_error <-c()
    for (m in names(models)){
        model <- models[[m]]
        if (is.null(model$error)){
            models_f[[m]] <- model$result
        }
        else {
            models_error[[m]] <- model
        }
    }
    return (models_f)
}



In [15]:
# filter down models
models <- filter_errored_models(models)
length(models)

[1] 670

In [16]:
# lets see what models look like
models[1]

$`chow_fast-HPF__01`

Family: gaussian 
Link function: identity 

Formula:
ME ~ get(group_col)
Total model degrees of freedom 2 

REML score: -2.917315     


In [17]:
# summarize models
plan(multisession, workers = 20)
summaries <- future_map(models, ~summary(.x))

In [18]:
length(summaries)

[1] 670

In [19]:
# make a vector of summaries
mgcv_summaries_f <- c()
for (s in names(summaries)){
    summary <- summaries[[s]]
    p.pv <- summary$p.pv[[2]]
    mgcv_summaries_f[[s]] <-summary
}

In [20]:
# save vector just in case
saveRDS(mgcv_summaries_f, summaries_path) 

In [21]:
# save models
saveRDS(models, models_path)

In [22]:
# data structure for export
ym_export <- c()
for (n in names(mgcv_summaries_f)){
    s <- mgcv_summaries_f[[n]]
    pval <- s$p.pv[2]
    coef <- s$p.coeff[2]
    ym_export[[n]] <- list('pvalue' = pval, "coef" = coef)
}

In [23]:
# what does it look like
ym_export

$`chow_fast-HPF__01`
$`chow_fast-HPF__01`$pvalue
get(group_col)fast 
         0.6672134 

$`chow_fast-HPF__01`$coef
get(group_col)fast 
       -0.03878617 


$`chow_fast-HPF__02`
$`chow_fast-HPF__02`$pvalue
get(group_col)fast 
         0.4306234 

$`chow_fast-HPF__02`$coef
get(group_col)fast 
       -0.05514482 


$`chow_fast-HPF__03`
$`chow_fast-HPF__03`$pvalue
get(group_col)fast 
         0.7202889 

$`chow_fast-HPF__03`$coef
get(group_col)fast 
       -0.02061315 


$`chow_fast-HPF__04`
$`chow_fast-HPF__04`$pvalue
get(group_col)fast 
         0.9042474 

$`chow_fast-HPF__04`$coef
get(group_col)fast 
      -0.001844766 


$`chow_fast-HPF__05`
$`chow_fast-HPF__05`$pvalue
get(group_col)fast 
         0.7930039 

$`chow_fast-HPF__05`$coef
get(group_col)fast 
        -0.0116775 


$`chow_fast-HPF__06`
$`chow_fast-HPF__06`$pvalue
get(group_col)fast 
         0.4450417 

$`chow_fast-HPF__06`$coef
get(group_col)fast 
        0.02404289 


$`chow_fast-HPF__07`
$`chow_fast-HPF__07`$pvalue
get(group_col)fast 
         0.5960894 

$`chow_fast-HPF__07`$coef
get(group_col)fast 
       -0.02420671 


$`chow_fast-HPF__08`
$`chow_fast-HPF__08`$pvalue
get(group_col)fast 
         0.6405049 

$`chow_fast-HPF__08`$coef
get(group_col)fast 
       -0.04956022 


$`chow_fast-HPF__09`
$`chow_fast-HPF__09`$pvalue
get(group_col)fast 
         0.3848464 

$`chow_fast-HPF__09`$coef
get(group_col)fast 
       -0.03675106 


$`chow_fast-HPF__10`
$`chow_fast-HPF__10`$pvalue
get(group_col)fast 
         0.9031776 

$`chow_fast-HPF__10`$coef
get(group_col)fast 
      -0.008014081 


$`chow_fast-HPF__11`
$`chow_fast-HPF__11`$pvalue
get(group_col)fast 
         0.9536518 

$`chow_fast-HPF__11`$coef
get(group_col)fast 
       0.002885071 


$`chow_fast-HPF__12`
$`chow_fast-HPF__12`$pvalue
get(group_col)fast 
         0.9131015 

$`chow_fast-HPF__12`$coef
get(group_col)fast 
      -0.002231982 


$`chow_fast-HPF__13`
$`chow_fast-HPF__13`$pvalue
get(group_col)fast 
        0.04964038 

$`chow_fast-HPF__13`$coef
get(group_col)fast 
       -0.02924081 


$`chow_fast-HPF__14`
$`chow_fast-HPF__14`$pvalue
get(group_col)fast 
         0.8609715 

$`chow_fast-HPF__14`$coef
get(group_col)fast 
       -0.01346729 


$`chow_fast-HPF__15`
$`chow_fast-HPF__15`$pvalue
get(group_col)fast 
         0.7886745 

$`chow_fast-HPF__15`$coef
get(group_col)fast 
      -0.008036801 


$`chow_fast-HPF__16`
$`chow_fast-HPF__16`$pvalue
get(group_col)fast 
         0.1520973 

$`chow_fast-HPF__16`$coef
get(group_col)fast 
         0.1799582 


$`chow_fast-HPF__17`
$`chow_fast-HPF__17`$pvalue
get(group_col)fast 
         0.1260351 

$`chow_fast-HPF__17`$coef
get(group_col)fast 
       -0.02278862 


$`chow_fast-HPF__18`
$`chow_fast-HPF__18`$pvalue
get(group_col)fast 
         0.1806729 

$`chow_fast-HPF__18`$coef
get(group_col)fast 
         0.1285865 


$`chow_fast-HPF__19`
$`chow_fast-HPF__19`$pvalue
get(group_col)fast 
         0.2840156 

$`chow_fast-HPF__19`$coef
get(group_col)fast 
        0.01849331 


$`chow_fast-HPF__20`
$`chow_fast-HPF__20`$pvalue
get(group_col)fast 
         0.4712862 

$`chow_fast-HPF__20`$coef
get(group_col)fast 
        0.03130818 


$`chow_fast-HPF__21`
$`chow_fast-HPF__21`$pvalue
get(group_col)fast 
         0.6593358 

$`chow_fast-HPF__21`$coef
get(group_col)fast 
         0.0101919 


$`chow_fast-HPF__22`
$`chow_fast-HPF__22`$pvalue
get(group_col)fast 
         0.6976282 

$`chow_fast-HPF__22`$coef
get(group_col)fast 
       -0.02554278 


$`chow_fast-HPF__23`
$`chow_fast-HPF__23`$pvalue
get(group_col)fast 
         0.3825608 

$`chow_fast-HPF__23`$coef
get(group_col)fast 
        -0.0424667 


$`chow_fast-HPF__24`
$`chow_fast-HPF__24`$pvalue
get(group_col)fast 
         0.5409044 

$`chow_fast-HPF__24`$coef
get(group_col)fast 
        0.02951245 


$`chow_fast-HPF__25`
$`chow_fast-HPF__25`$pvalue
get(group_col)fast 
        0.09014498 

$`chow_fast-HPF__25`$coef
get(group_col)fast 
       -0.02822796 


$`chow_fast-HPF__26`
$`chow_fast-HPF__26`$pvalue
g

In [24]:
# convert to tibble and save as csv
df <- ym_export %>%
  imap(function(value, name) {
    tibble(
      test_name = name,
      coef = value$coef,
      pvalue = value$pvalue
    )
  }) %>%
  bind_rows() %>% 
  arrange(pvalue) %>% 
  mutate(pvalue_bh = p.adjust(pvalue, method = "BH")) %>% 
  arrange(pvalue_bh) %>%
  separate('test_name', into = c('comparison', 'module'), sep='-') %>%
  mutate(comparison = str_replace(comparison, fixed('_'), '.vs.'))

df %>% head(10)
df %>% write_csv(significant_modules)

comparison,module,coef,pvalue,pvalue_bh
<chr>,<chr>,<dbl>,<dbl>,<dbl>
chow.vs.fast,HY__50,0.04283113,0.0003678630,0.2464682
chow.vs.fast,HY__71,-0.02796588,0.0009602221,0.3216744
chow.vs.HFD,HY__54,0.12215175,0.0018235949,0.3363571
chow.vs.HFD,HPF__17,-0.04071771,0.0022904364,0.3363571
chow.vs.HFD,HPF__25,-0.05090986,0.0025101275,0.3363571
chow.vs.fast,SC__32,-0.06560489,0.0064002652,0.4830956
chow.vs.fast,IC__08,0.07363731,0.0066662706,0.4830956
chow.vs.HFD,HPF__05,-0.12785654,0.0067350821,0.4830956
chow.vs.fast,IC__17,-0.10352119,0.0077718854,0.4830956


In [25]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux

Matrix products: default
BLAS/LAPACK: /nfsdata/tools/anaconda/envs/nmq407/dmb/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_GB.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_GB.UTF-8        LC_COLLATE=en_GB.UTF-8    
 [5] LC_MONETARY=en_GB.UTF-8    LC_MESSAGES=en_GB.UTF-8   
 [7] LC_PAPER=en_GB.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_GB.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] lubridate_1.9.2             stringr_1.5.0              
 [3] purrr_1.0.1                 readr_2.1.4                
 [5] tibble_3.2.1                tidyverse_2.0.0            
 [7] furrr_0.3.1                 future_1.32.0              
 [9] fitdistrplus_1.1-11         sur